In [26]:
from sklearn.linear_model import Lasso
from sklearn.datasets import load_boston
import pickle
import requests
import pandas as pd
import numpy as np
import seaborn as sns

# Documentation of ML-API

## Idea

In our project `Immobilienpreisrechner` in the third semester we have built a Machine Learning model to predict housing prices as accurate as possible. One task of this project was to implement an API for our model as well. Our API was not strucutured very well and it did not allow to pre-process the data same as the data we used to build and train our model. We did not a have a clear data pipeline for our model. In this small project for the study course web data collection I wanted to fix this issue and implement a basis to make a possible roll-out of the model possible.

## Description of the Service

In my opinion this service provides a framework for me in the fututer to implement a machine learning model and roll it out for customer or a colleague.
The API provides all relevant operations like predictions, scoring for customers as well as for a maintainer with the methods to update the model parameters and deletion of the model.


**Implement CI CD with this -> https://www.youtube.com/watch?v=AZMQVI6Ss64&ab_channel=AlejandroMagnitopic**

### Load Sample Data

Now we are loading the Sample data as defined above in our RAM [(Pedregosa et al., 2011)](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html).

In [27]:
X, y = load_boston(return_X_y=True)

### Create new Model Endpoint

Endpoint: `\create_model`

Creates a new model with a fitting id within the src folder. Then the id of the new model will be returned for further usage.
Because it is not possible to upload a python object to the API it is currently only possible to adapt an existing model. For example if there are only Lasso Regression models stored in src folder you can only adapt the lasso model.

In [28]:
# The URL of our API 
base_url = 'http://simonstaehli.pythonanywhere.com'

In [29]:
new_model_params = {'alpha': 2}

In [30]:
# Send request
response = requests.put(url=base_url+'/create_model', json=new_model_params)
print('HTTP-Statuscode:', response.status_code)

HTTP-Statuscode: 200


In [31]:
response.json()

{'model_id': 2}

### Prediction Endpoint

Endpoint : `\predict`

First endpoint of the API, which we will check, is the prediction endpoint. With this endpoint you can send data to the API with POST method. 
We will convert our numpy array to a dataframe. This will make it easier to convert to JSON datatype which is needed for the API.

In [32]:
# Create dataframe to convert it to json
data = pd.DataFrame(X)
data.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.9,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14


In [33]:
# Convert to JSON
data = data.to_json(orient='records')

In [34]:
# Send request
response = requests.post(url=base_url+'/predict/2', json=data)
print('HTTP-Statuscode:', response.status_code)

HTTP-Statuscode: 200


In [35]:
print('Predictions : \n', response.json()[:5])

Predictions : 
 [30.997539182131007, 25.776817360601793, 29.986014494727087, 29.517998131638052, 28.032489994804962]


### Score Endpoint

Endpoint: `\score`

Next we will test the Scoring endpoint of the API. The scoring endpoint returns the score of the model reached by the uploaded data. Therefore we have to send the X and y data encoded as JSON.

In [11]:
y_ = pd.DataFrame(y)
y_ = y_.to_json(orient='records')

In [12]:
data_ = dict(X=data, y=y_)

In [13]:
response = requests.post(url=base_url+'/score/2', json=data_)
print('HTTP-Statuscode:', response.status_code)

HTTP-Statuscode: 200


In [14]:
print('R2-Score: ',response.json())

R2-Score:  0.6825842212709925


### Model Coefficients Endpoint

Endpoint: `\model_params`

This endpoint can be reached with a GET requests and returns the model parameters. The model parameters depend on the model used. It is expected that this changes from model to model. In accordance with that the model parameters have to be changed.

In [36]:
response = requests.get(url=base_url + '/model_params/2')
print('HTTP-Statuscode:', response.status_code)

HTTP-Statuscode: 200


In [37]:
model_coef = response.json()
model_coef

{'alpha': 2,
 'coef_': [-0.06343729004514066,
  0.04916466550764739,
  -0.0,
  0.0,
  -0.0,
  0.9498106999845143,
  0.020909514944737546,
  -0.6687900023707882,
  0.26420643097453383,
  -0.01521158979163473,
  -0.7229663585199505,
  0.00824703348549421,
  -0.7611145367697878],
 'copy_X': True,
 'dual_gap_': 4.238246337692544,
 'fit_intercept': True,
 'intercept_': 41.05693374499337,
 'l1_ratio': 1.0,
 'max_iter': 1000,
 'n_features_in_': 13,
 'n_iter_': 42,
 'normalize': False,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

### Update Model Endpoint

Endpoint: `/update_model`

This endpoint can be reached with a put request. With this requests you give the model new model parameters. As well as the method above this need to be changed as the model parameters strongly depend on the model. However it makes sense to change the model parameters if the model need to be retrained with new data for instance time series data.

In [18]:
# Update model parameters
response = requests.put(url=base_url+'/update_model/2', json={'params': np.random.uniform(size=len(model.coef_)).tolist()})
print('HTTP-Statuscode:', response.status_code)

NameError: name 'model' is not defined

In [24]:
# Print response
response.json()

'Parameters updated successfully.'

In [19]:
# Check if model parameters have changed by requesting the model parameters
response = requests.get(url=base_url + '/model_params/2')

In [21]:
model_coef = response.json()

print('New Model Parameters: \n', model_coef)
print('Old Model Parameters: \n', model.coef_)

New Model Parameters: 
 {'alpha': 0.5, 'coef_': [-0.06343729004514066, 0.04916466550764739, -0.0, 0.0, -0.0, 0.9498106999845143, 0.020909514944737546, -0.6687900023707882, 0.26420643097453383, -0.01521158979163473, -0.7229663585199505, 0.00824703348549421, -0.7611145367697878], 'copy_X': True, 'dual_gap_': 4.238246337692544, 'fit_intercept': True, 'intercept_': 41.05693374499337, 'l1_ratio': 1.0, 'max_iter': 1000, 'n_features_in_': 13, 'n_iter_': 42, 'normalize': False, 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}


NameError: name 'model' is not defined

In comparisson to the old parameters we can see that the parameters have been changed now.

### Delete Model Endpoint

Endpoint: `\delete_model` 

As we have changed the model parameters the model is in fact not usable anymore. Ultimately we have to delete it. For this purpose I created a delete endpoint. 

In [38]:
response = requests.delete(url=base_url+ '/delete_model/2')
print('HTTP-Statuscode:', response.status_code)

HTTP-Statuscode: 200


In [39]:
print(response.json())

Model Deleted Successfully.
